In [21]:
using QuantumOptics
using BenchmarkTools
using LinearAlgebra

In [79]:
"""
    drazin(H, J; rho_ss = steadystate.eigenvector(H, J))
    
Calculate the Drazin inverse of a Liouvillian defined by the Hamiltonian H and jump operators J.
 
# Arguments
* `H`: Arbitrary operator specifying the Hamiltonian.
* `J`: Vector containing all jump operators which can be of any arbitrary
         operator type.
* `rho_ss`: Density matrix specifying the steady-state of the Liouvillian. By default, it is found through steadystate.eigenvector. 
         For large matrices the steady-state should be provided, as the best steady-state solver could vary.
 """    
    function drazin(H::AbstractOperator, J; ρss = steadystate.eigenvector(H, J))
        d = length(H)
        # Vectorizing the steady-state and the identity
        vId = reshape(Matrix(identityoperator(H).data), d)'
        vss = reshape(Matrix(ρss.data), d)
        # vectorized liouvillian
        vL = Matrix(liouvillian(H, J).data)
        # Liouville space's identity
        IdL = Matrix{ComplexF64}(I, d, d)
        # We introduce Q, which projects any vector in the complement of the kernel o L
        Q = IdL - vss*vId
        # The Drazin inverse is computed by projecting the Moore-Penrose pseudo-inverse, computed using pinv.
        LD = Q*pinv(vL)*Q
        return LD
    end

drazin

In [104]:
""" 

    drazin_apply() 
Calculates the vector resulting from the Drazin inverse being applied to another vector. 

# Arguments 
* `H`: Arbitrary operator specifying the Hamiltonian.
* `J`: Vector containing all jump operators which can be of any arbitrary
    operator type.
* `ρss ` : steady state of the system as a density matrix
* `α` : input state as a density matrix

"""
function drazin_apply(H, J, αvec, ρss = steadystate.master(H,J)[2][1])
    ## Constructing the matrix 

  # constructing the liouvillian from the Hamiltonian and the jump operators 
  L = Matrix(liouvillian(H, J;).data)
  
  # constructing unitmatrix to append to liouvillian
  unitrow = vec(identityoperator(basis).data)'
  
  # constructing the left hand side consiting of the liouvillian and the unit matrix row 
  lhs = cat(L, unitrow; dims = 1)

  ## Constructing the right hand side 

  # vectorizing the steady state
  ρssvec = vec(Matrix(ρss.data))
  
  # constructing the right hand side of the linear system 
  rhs = append!(αvec - ρssvec .* (unitrow * αvec), 0)

  ## returning the result 
  
  return lhs\rhs

end 

drazin_apply

In [89]:
## Example Quantum Sytem to try the function out 
basis = FockBasis(10);
# creation and annihilation operators of the cavity
a = destroy(basis);
at = create(basis);
H = at*a + 2*at*at*a*a;
# Jump operators and their rates to simulate coupling to a bath 
J = [a];
#Jt = dagger(a)

1-element Vector{Operator{FockBasis{Int64}, FockBasis{Int64}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}}:
 Operator(dim=11x11)
  basis: Fock(cutoff=10)sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], ComplexF64[1.0 + 0.0im, 1.4142135623730951 + 0.0im, 1.7320508075688772 + 0.0im, 2.0 + 0.0im, 2.23606797749979 + 0.0im, 2.449489742783178 + 0.0im, 2.6457513110645907 + 0.0im, 2.8284271247461903 + 0.0im, 3.0 + 0.0im, 3.1622776601683795 + 0.0im], 11, 11)

In [90]:
ρα = coherentstate(basis,1) ⊗ dagger(coherentstate(basis,1)); 
αvec = vec(ρα.data);
ρss = steadystate.master(H,J)[2][1];


In [105]:
diff = drazin_apply(H, J, αvec, ρss) - drazin(H, J; ρss) * αvec

121-element Vector{ComplexF64}:
 -1.4432899320127035e-15 + 3.7484682552881483e-16im
   5.967448757360216e-16 - 1.6653345369377348e-16im
   2.246466901389965e-16 + 5.551115123125783e-17im
 -2.2551405187698492e-17 + 4.336808689942018e-17im
  -8.058332646998512e-17 - 3.903127820947816e-17im
  -6.076953176781252e-17 - 1.4094628242311558e-17im
 -3.3771203607028955e-18 - 2.656295322589486e-18im
 -2.7810209240726816e-18 + 4.119968255444917e-18im
  3.6569059310928315e-18 - 8.453388813597917e-19im
  1.4386589911318274e-18 - 1.1443415117405598e-18im
                         ⋮
   5.376574719921174e-19 - 1.1722935989999517e-18im
   5.565999080195817e-19 - 8.819730563285402e-19im
 -2.4652612688428143e-18 + 1.7926922933045233e-18im
   5.419563296355554e-19 + 1.356258567231745e-18im
  3.0583381875147594e-18 + 1.2417569181197047e-18im
 -1.5682252535596835e-18 + 6.204185466392729e-19im
  3.2255852151813967e-18 + 2.6604990544624803e-18im
  3.2850237256624665e-20 - 2.7576547261292837e-20im
  1.2907144298

In [106]:
norm(diff)

1.785880842508949e-15